KoGPT2 파인튜닝 https://github.com/SKT-AI/KoGPT2

### 구글 마운트

In [2]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# 구글 드라이브 경로 설정
%cd /content/drive/MyDrive/텍스트 프로젝트
%pwd

Mounted at /content/drive
/content/drive/MyDrive/텍스트 프로젝트


'/content/drive/MyDrive/텍스트 프로젝트'

### 라이브러리 설치

In [3]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.0 MB/s eta 0:00:00


### 라이브러리 불러오기

In [7]:
import pandas as pd
import numpy as np

# for padding
# from tensorflow.keras.preprocessing.sequence import pad_sequences

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW
from tqdm import tqdm

import time
import datetime

### 모델 테스트


In [ ]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
bos_token='</s>', eos_token='</s>', unk_token='<unk>',
pad_token='<pad>', mask_token='<mask>')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
tokenizer.tokenize("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o")

['▁안녕',
 '하',
 '세',
 '요.',
 '▁한국어',
 '▁G',
 'P',
 'T',
 '-2',
 '▁입',
 '니다.',
 '😤',
 ':)',
 'l^o']

In [ ]:
import torch
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

In [ ]:
text = '근육이 커지기 위해서는'
input_ids = tokenizer.encode(text, return_tensors='pt')
gen_ids = model.generate(input_ids,
                        max_length=128,
                        repetition_penalty=2.0,
                        pad_token_id=tokenizer.pad_token_id,
                        eos_token_id=tokenizer.eos_token_id,
                        bos_token_id=tokenizer.bos_token_id,
                        use_cache=True)
generated = tokenizer.decode(gen_ids[0])
print(generated)

근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.
특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다.
또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다.
아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다.
운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다.
운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다.
운동을


### 사용자 정의 함수

In [ ]:
import re

# 곡제목 전처리
def preprocess(sentences):
    lst = []
    for sentence in sentences:
        # \n과 \t를 제거한다
        sentence = re.sub('\t', '', sentence)
        sentence = re.sub('\n', '', sentence)
        # 문장 양옆의 띄어쓰기를 지운다
        sentence = sentence.strip()

        lst.append(sentence)
    return lst

### 데이터프레임 생성 - 장르, 곡제목

In [ ]:
rank = pd.read_csv('data/순위.csv')
album_info = pd.read_csv('data/앨범정보.csv')
genre_steady = pd.read_csv('data/장르별 스테디셀러.csv')

In [ ]:
rank.tail(3)

,Unnamed: 0,year,rank,song_name,artist,likes,album_id,song_id
1297,1297,2010년,98,이별이 온다,에이트,"6,548",882152,2673348
1298,1298,2010년,99,19금 부치지 못한 편지,DJ DOC,"16,031",968252,2783256
1299,1299,2010년,100,Bye Bye Bye,박화요비,"12,659",904355,2697459


In [ ]:
album_info.tail(3)

,Unnamed: 0,AlbumID,Release_date,Genre,Publisher,Agency
1007,497,10020654,2016.12.03,"국내드라마, 발라드, 랩/힙합","지니뮤직, Stone Music Entertainment","Stone Music Entertainment, 화앤담픽쳐스"
1008,498,10020656,2016.12.05,R&B/Soul,지니뮤직,Stone Music Entertainment
1009,499,911357,2010.06.03,"댄스, 발라드",카카오엔터테인먼트,스타쉽 엔터테인먼트


album_info와 rank 데이터프레임의 album_id를 이용해

장르, 곡제목으로 이뤄진 데이터프레임 만들기

In [ ]:
tmp1 = album_info[['AlbumID', 'Genre']]
tmp1.columns = ['album_id', 'genre']
tmp1.tail(3)

,album_id,genre
1007,10020654,"국내드라마, 발라드, 랩/힙합"
1008,10020656,R&B/Soul
1009,911357,"댄스, 발라드"


In [ ]:
tmp2 = rank[['album_id', 'song_name']]
tmp2.tail(3)

,album_id,song_name
1297,882152,이별이 온다
1298,968252,19금 부치지 못한 편지
1299,904355,Bye Bye Bye


In [ ]:
rank_df = pd.merge(tmp1, tmp2, on=['album_id'])
print(tmp1.shape)
print(tmp2.shape)
print(rank_df.shape)
rank_df.tail(3)

(1010, 2)
(1300, 2)
(1300, 3)


,album_id,genre,song_name
1297,10020654,"국내드라마, 발라드, 랩/힙합",Stay With Me
1298,10020656,R&B/Soul,저 별
1299,911357,"댄스, 발라드",Push Push (푸시 푸시)


genre_steady 데이터프레임의 gnrCode를 장르 이름으로 변경하기

* 'GN0100': 발라드
* 'GN0200': 댄스
* 'GN0300': 랩/힙합
* 'GN0400': R&B/Soul
* 'GN0500': 인디음악
* 'GN0600': 록/메탈
* 'GN0700': 트로트
* 'GN0800': 포크/블루스

In [ ]:
genre_steady

,gnrCode,rank,song_name,artist,album_id,song_id
0,GN0100,1,내 손을 잡아,아이유,1286252,3414749
1,GN0100,2,Timeless,SG 워너비,399659,1944399
2,GN0100,3,봄 사랑 벚꽃 말고,HIGH4 (하이포),2248659,4579468
3,GN0100,4,희재,성시경,28985,418168
4,GN0100,5,라라라,SG 워너비,376502,1854856
...,...,...,...,...,...,...
2881,GN0800,271,데미안,심규선 (Lucia),2257294,4653838
2882,GN0800,272,어느 날,10CM,2150728,3909667
2883,GN0800,273,3X4,가을방학,1035872,2960596
2884,GN0800,274,"우정, 그 씁쓸함에 대하여",10CM,1166956,3136469


In [ ]:
genre_steady[['album_id', 'gnrCode', 'song_name']]

,album_id,gnrCode,song_name
0,1286252,GN0100,내 손을 잡아
1,399659,GN0100,Timeless
2,2248659,GN0100,봄 사랑 벚꽃 말고
3,28985,GN0100,희재
4,376502,GN0100,라라라
...,...,...,...
2881,2257294,GN0800,데미안
2882,2150728,GN0800,어느 날
2883,1035872,GN0800,3X4
2884,1166956,GN0800,"우정, 그 씁쓸함에 대하여"


In [ ]:
genre_dict = {'GN0100': '발라드'
,'GN0200': '댄스'
,'GN0300': '랩/힙합'
,'GN0400': 'R&B/Soul'
,'GN0500': '인디음악'
,'GN0600': '록/메탈'
,'GN0700': '트로트'
,'GN0800': '포크/블루스'}

genre_dict['GN0100']

'발라드'

In [ ]:
lst = []
for album_id, genre, song in genre_steady[['album_id', 'gnrCode', 'song_name']].to_numpy():
  lst.append([album_id, genre_dict[genre], song])

In [ ]:
columns = ['album_id', 'genre', 'song_name']
steady_df = pd.DataFrame(lst, columns=columns)

In [ ]:
print(steady_df.shape)
steady_df.tail(3)

(2886, 3)


,album_id,genre,song_name
2883,1035872,포크/블루스,3X4
2884,1166956,포크/블루스,"우정, 그 씁쓸함에 대하여"
2885,867153,포크/블루스,Healing


rank_df, steady_df 데이터프레임을 합친다. (album_id 중복 제거)

In [ ]:
genre_song_df = pd.concat([rank_df, steady_df]).reset_index(drop=True)
print(genre_song_df.shape)
genre_song_df.tail(3)

(4186, 3)


,album_id,genre,song_name
4183,1035872,포크/블루스,3X4
4184,1166956,포크/블루스,"우정, 그 씁쓸함에 대하여"
4185,867153,포크/블루스,Healing


In [ ]:
genre_song_df = genre_song_df.drop_duplicates(['album_id'])
genre_song_df = genre_song_df.reset_index(drop=True)
print(genre_song_df.shape)
genre_song_df.tail(3)

(2547, 3)


,album_id,genre,song_name
2544,2641481,포크/블루스,"카라멜마끼아또 (With. 장규철,이시은)"
2545,2257294,포크/블루스,데미안
2546,2150728,포크/블루스,어느 날


In [ ]:
genre_song_df.to_csv('genre_song.csv', index=False)

### 데이터 불러오기

In [ ]:
genre_song_df = pd.read_csv('genre_song.csv')
genre_song_df['song_name'] = preprocess(genre_song_df['song_name'])

In [ ]:
genre_song_df['song_name']

0       이 밤을 빌려 말해요 (바른연애 길잡이 X 10CM)
1                       옥탑방 (Rooftop)
2              이게 사랑이 아니면 (Feat. 에일리)
3                                 흩어져
4                         KNOCK KNOCK
                    ...              
2542          가끔 미치도록 네가 안고 싶어질 때가 있어
2543                           니가 궁금해
2544          카라멜마끼아또 (With. 장규철,이시은)
2545                              데미안
2546                             어느 날
Name: song_name, Length: 2547, dtype: object

In [ ]:
# 장르가 comma(,)로 여러개 구분되있는 경우 맨 처음만 장르로 사용
genre_song_df['genre'] = [genre.split(', ')[0] for genre in genre_song_df['genre'].to_list()]

In [ ]:
# 장르 종류
genre_song_df['genre'].unique()

array(['발라드', '록/메탈', '랩/힙합', '댄스', 'R&B/Soul', '인디음악', '국내드라마', '포크/블루스',
       '일렉트로니카', '성인가요/트로트', '트로트'], dtype=object)

In [ ]:
data_lst = []
for genre, song in genre_song_df[['genre', 'song_name']].to_numpy():
  data_lst.append(genre+': '+song)

print(len(data_lst))
data_lst[:10]

2547


['발라드: 이 밤을 빌려 말해요 (바른연애 길잡이 X 10CM)',
 '록/메탈: 옥탑방 (Rooftop)',
 '랩/힙합: 이게 사랑이 아니면 (Feat. 에일리)',
 '발라드: 흩어져',
 '댄스: KNOCK KNOCK',
 '록/메탈: 스물다섯, 스물하나',
 'R&B/Soul: GANADARA (Feat. 아이유)',
 '발라드: 하늘색 약속',
 '댄스: 미치겠어',
 '랩/힙합: Boogie On & On']

### DataLoader

In [ ]:
print('[장르: 곡제목]의 최대 길이:', max(len(data) for data in data_lst))

[장르: 곡제목]의 최대 길이: 75


In [ ]:
len(data_lst)

2547

In [ ]:
class SongDataset(Dataset):
    def __init__(self, tokenizer, data):

        inputs = tokenizer(
            data,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=80,
        )

        self.item = inputs["input_ids"]
        self.attention_mask = inputs["attention_mask"]

        self.length = len(self.item)

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        return self.item[idx], self.attention_mask[idx]

In [ ]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                pad_token='<pad>', mask_token='<mask>')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
tokenizer.tokenize('가나다')

['▁가', '나다']

In [ ]:
batch_size=32

train_data = SongDataset(tokenizer, data_lst)
train_dataloader = DataLoader(train_data, batch_size=batch_size)

In [ ]:
# 데이터셋 잘 만들어졌는지 확인
train_data.__getitem__(3)

(tensor([43760,   401, 17805,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3]),
 tensor([1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]))

In [ ]:
len(train_dataloader)

80

In [ ]:
# 데이터로드 잘 만들어졌는지 확인
dataiter = iter(train_dataloader)
item, attention_mask = next(dataiter)

In [ ]:
item

tensor([[43760,   401,  9018,  ...,     3,     3,     3],
        [19969,   390,  7521,  ...,     3,     3,     3],
        [42998,   390,  8816,  ...,     3,     3,     3],
        ...,
        [47647,   401, 14659,  ...,     3,     3,     3],
        [47647,   401, 46260,  ...,     3,     3,     3],
        [47647,   401, 27895,  ...,     3,     3,     3]])

In [ ]:
attention_mask

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

### GPU 사용

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


### 모델 만들기

In [ ]:
import torch
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2').to(device)

In [ ]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))  # hh:mm:ss

In [ ]:
load_data = torch.load("model_kogpt2/genre_song/kogpt2_fine-tunning_22.pt")

In [ ]:
load_data.keys()

dict_keys(['epoch', 'loss', 'model', 'optimizer'])

In [ ]:
#load_epoch = 0
load_epoch = load_data['epoch']
load_epoch

22

In [ ]:
load_loss = load_data['loss']
load_loss

0.07684741169214249

In [ ]:
# 저장한 모델 불러오기
# 모델 로드
model.load_state_dict(load_data['model'])

<All keys matched successfully>

In [ ]:
# cuda 메모리 지우기
import torch
torch.cuda.empty_cache()

import gc
gc.collect()

135

In [ ]:
def generate(keyword):
    input_ids = tokenizer.encode(keyword, return_tensors='pt')
    input_ids = input_ids.to(device)
    gen_ids = model.generate(input_ids,
                        max_length=128,
                        repetition_penalty=2.0,
                        pad_token_id=tokenizer.pad_token_id,
                        eos_token_id=tokenizer.eos_token_id,
                        bos_token_id=tokenizer.bos_token_id,
                        use_cache=True)
    generated = tokenizer.decode(gen_ids[0])
    return generated

In [ ]:
keywords = genre_song_df['genre'].unique()

# 학습 전 generate 함수 동작 테스트
for keyword in keywords:
    #keyword = keyword.to(device)
    gen = generate(keyword)
    gen_print = gen.split('\n')[0:2]
    print(f"{keyword} : {gen_print}")

발라드 : ['발라드 등 다양한 장르의 음악을 선보인다.', '이번 공연은 지난해 11월부터 올 1월까지 총 4회에 걸쳐 진행됐으며 모두 1만5000여명의 관객을 동원했다.']
록/메탈 : ['록/메탈기어 솔리드 4의 후속작.</d> #20180923 오늘의 아침<unk>', '오늘은 어제보다 더 춥습니다.']
랩/힙합 : ['랩/힙합, 섹시댄스, 록밴드 등 다양한 장르의 음악을 선보인다.', '이번 공연은 지난해 11월부터 올 1월까지 총 4회에 걸쳐 진행됐으며 모두 1만5000여명의 관객을 동원했다.']
댄스 : ['댄스 등 다양한 장르의 음악을 선보인다.', '이번 공연은 지난해 11월부터 올 1월까지 총 4회에 걸쳐 진행됐으며 관객들의 뜨거운 호응을 얻었다.']
R&B/Soul : ["R&B/Soul, SK텔레콤 등 국내 주요 대기업과 함께 '2018 대한민국 브랜드대상'을 수상했다.", '이번 수상은 지난해 11월부터 올 1월까지 진행된 ‘대한민국 소비자 대상’ 조사에서 총 423개 기업이 응모한 가운데 최종 선정됐다.']
인디음악 : ['인디음악은 물론 재즈, 팝 등 다양한 장르의 음악을 선보인다.', "이번 공연에서는 '2018 평창동계올림픽' 공식 후원사인 SK텔레콤과 함께하는 특별공연도 마련된다."]
국내드라마 : ['국내드라마 ‘태양의 후예’ 촬영지로도 유명하다.', '이번 공연은 지난해 11월부터 올 1월까지 총 4회에 걸쳐 진행됐다.']
포크/블루스 : ['포크/블루스', '2016년 1월 1일, #1 ( 2016 ) - 1화부터 등장.</d> "그럼에도 불구하고 나는 이 일을 계속할 수 있을 거야!"']
일렉트로니카 : ['일렉트로니카, 닌텐도 스위치, 소니, 파나소닉 등 일본 대표 전자제품들이 대거 전시된다.', '특히 이번 전시회에서는 세계 최초로 공개되는 ‘파워풀한 디자인’을 통해 관람객들의 시선을 사로잡을 예정이다.']
성인가요/트로트 : ['성인가요/트로트 가수들', "이런저렇게 많은 사람들이 모여있는 곳에서, 그 중에서도 특히

In [ ]:
# 학습 모드
model.train()
min_loss = load_loss
len_size = len(genre_song_df['genre'].unique())
epoch_num = 10

for epoch in range(load_epoch + 1, epoch_num+load_epoch):
    print('======== Epoch {:} / {:} ========'.format(epoch, epoch_num+load_epoch))
    t0 = time.time()
    total_loss = 0

    for input_text, attention_mask in tqdm(train_dataloader):
        input_tensor = input_text.to(device)
        attention_mask = attention_mask.to(device)
        # print(input_tensor)
        # print(input_tensor.shape)

        # input과 label이 같다.
        outputs = model(input_tensor, labels=input_tensor, attention_mask=attention_mask)
        # print('outputs:', outputs)
        loss = outputs[0]

        # cpu로 변경하기
        input_tensor.cpu()

        optimizer.zero_grad()
        model.zero_grad()
        loss.backward()
        optimizer.step()

    now_loss = outputs[0].item()
    print(f"epoch {epoch} loss {now_loss:0.2f}")
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

    # loss가 min_loss보다 작을 경우
    if now_loss < min_loss:
        min_loss = now_loss
        # 모델 저장
        torch.save({'epoch':epoch,
                    'loss':min_loss,
                    'model':model.state_dict(),
                    'optimizer':optimizer.state_dict()},
                   f"model_kogpt2/genre_song/kogpt2_fine-tunning_{epoch}.pt")



    # 문장 출력해보기
    # 랜덤으로 중복되지 않게 장르를 뽑아서 출력된 문장을 확인해본다.
    numbers = np.random.choice(range(0,11), 3, replace=False)
    keywords = genre_song_df['genre'].unique()[numbers]
    for keyword in keywords:
        gen = generate(keyword)
        gen_print = gen.split('\n')[0:2]
        print(f"{keyword} : {gen_print}")

======== Epoch 23 / 32 ========


100%|██████████| 80/80 [00:50<00:00,  1.60it/s]


epoch 23 loss 0.09
  Training epcoh took: 0:00:50
록/메탈 : ['록/메탈: 사랑했지만<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>ath로이선 (Room)<pad><pad><pad><pad>nal of Bravo, Lonely malrodies, Paak이 새어지다 (12th BI G NEVERNARAYDASKorea은거송희진청춘꽃피는 오골계 & 2nictoast Every, Verbian Rising For You Angel Meet Say With 서인']
일렉트로니카 : ["일렉트로니카: 아낀다<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>ath육을 수 있을까요... (Goodbye)<pad><pad><pad><pad><pad><pad><pad><pad>nalormoftop)<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>three Betrays아로로이드를 제거한 뒤 다시 포장재에서루어린듯 비에 젖어도단 이유기'바른걸음걸음표 Glooming back Edit. Sunrise oftoch-Freship"]
댄스 : ["댄스: Rainism (Clean Ver.)<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>)<pad><pad><pad><pad><pad>athernights Soulfekese)<pad><pad><pad><pad><pad>naloprody Get In Lovebaby, Bonush o

100%|██████████| 80/80 [00:48<00:00,  1.67it/s]


epoch 24 loss 0.08
  Training epcoh took: 0:00:48
일렉트로니카 : ['일렉트로니카: 19금전화기를 꺼놔 (Turn Off Your Phone)<pad><pad><pad><pad><pad><pad><pad><pad><pad>ath)<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>ationate of구간사)<pad><pad><pad><pad><pad>nalogo Babyat, 박완규&정일우 & 조권분여일)<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>)’인 걸음이새로이미선)’을 걸고 노력해 보고싶어요! 라면법<pad>']
트로트 : ['트로트: 사랑아 가자<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>athrodue Route by GRAY)<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>ast 인디고래 (Dance Ver.)<pad><pad><pad><pad><pad><pad><pad>ate of Babylon)<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>護셔널로네다음날까지기대해er)<pad><pad><pad><pad><pad><pad><pad><pad><pad>ealetationoftberry methodologe)<pad><pad><pad><pad><pad><pad>threezeatopret']
록/메탈 : ['록/메탈: 사랑한 후에<pad><pad><pad><pad><pad><pad><pad><pad>athroom (Feat. 

...

epoch 29 loss 0.09
  Training epcoh took: 0:00:50
성인가요/트로트 : ['성인가요/트로트: 이제 나만 믿어가<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>야만 해)<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>일)린스님과 함께<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>념에너블이 부착된 커피를 마시고잇어사)iplletether Romance goodby Monsterretty Babylon수거한 봉지개 (Feat. G-DRAGONELY 원곡가수분말고래부쓰리형아예)속에']
일렉트로니카 : ['일렉트로니카: 콩떡빙수<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>)년인 12월 1일이 2일이라고 하자11908번째 주말리무진 (Feat. 버벌진이스트, BI G NABLEY)<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad> 말머리날은 다정히 눈을 감고 서면역에서부터 60대 노부부까지)의미달기사)릭스폰트 (12개사 중 1개소녀편곡)’ 삽입부제목격자명령요청고']
포크/블루스 : ['포크/블루스: 사랑했지만<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>)<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>야만녕의 날씨에 (Main Ver.)<pad><pad><pad><pad><pad>쯤에서 너와 나 등만해berryse)<pad><pad><pad><p

100%|██████████| 80/80 [00:49<00:00,  1.61it/s]


epoch 30 loss 0.08
  Training epcoh took: 0:00:50
록/메탈 : ['록/메탈: 사랑앓이<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>ation<pad><pad><pad><pad><pad><pad>)<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>쯤 되면니 (에뛰드 복숭아송 원곡수월가도 삽입노래)의 멜로디로고백고개에서렉트 (12&2)<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>편지처럼<pad><pad>']
일렉트로니카 : ["일렉트로니카: 콩떡빙수<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>까지ine Love Blossomunion Meas You And Jehmy Koung Babylon)<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>ath)<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>쯤엔 너였어요 (D

100%|██████████| 80/80 [00:49<00:00,  1.60it/s]


epoch 31 loss 0.07
  Training epcoh took: 0:00:50
발라드 : ['발라드: 사랑<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>)<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad> 서면역에서<pad><pad><pad><pad><pad><pad>스인거죠면<pad><pad><pad>쯤에서니까요 (So What You Are성 Sense다예)<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>일)가기래림<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>thalistes into theseons For팽트<pad>']
성인가요/트로트 : ['성인가요/트로트: 찐이야<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>)<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad> 까지부 (Loftinesse)<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>thing bettersunnack Swears)<pad><pad><pad><pad><pad><pad><pad><pad><

In [ ]:
# 학습 후 generate 함수 동작 테스트
keywords = genre_song_df['genre'].unique()

for keyword in keywords:
    #keyword = keyword.to(device)
    gen = generate(keyword)
    gen_print = gen.split('\n')[0:2]
    print(f"{keyword} : {gen_print}")


발라드 : ["발라드: 너는 나 나는 나도<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>)<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>야만 해<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>thoulsShees안장념머리 (Don't Be Shots)<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>쯤면예고2)<pad><pad><pad>"]
록/메탈 : ['록/메탈: 사랑한 후에<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>) (Locing Story Measker)<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>o음이라도<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [ ]:
# 모델 로드
# model.load_state_dict(torch.load("model_kogpt2/genre_song/kogpt2_fine-tunning_14.pt"))

### 예측 테스트

In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [46]:
import torch
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2').to(device)

optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8)

load_data = torch.load("model_kogpt2/genre_song/kogpt2_fine-tunning_15.pt")


# 저장한 모델 불러오기
# 모델 로드
model.load_state_dict(load_data['model'])

# 옵티마이저 로드
optimizer.load_state_dict(load_data['optimizer'])

In [47]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                pad_token='<pad>', mask_token='<mask>')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [48]:
def generate(keyword):
    input_ids = tokenizer.encode(keyword, return_tensors='pt')
    input_ids = input_ids.to(device)
    gen_ids = model.generate(input_ids,
                        max_length=128,
                        repetition_penalty=2.0,
                        pad_token_id=tokenizer.pad_token_id,
                        eos_token_id=tokenizer.eos_token_id,
                        bos_token_id=tokenizer.bos_token_id,
                        use_cache=True)
    generated = tokenizer.decode(gen_ids[0])
    pad_idx = generated.find('<pad>')
    generated = generated[:pad_idx]
    return generated

In [49]:
keywords = ['발라드', '록/메탈', '랩/힙합', '댄스', 'R&B/Soul', '인디음악', '국내드라마', '포크/블루스', '일렉트로니카', '성인가요/트로트', '트로트']

for keyword in keywords:
    #keyword = keyword.to(device)
    gen = generate(keyword)
    print(f"{gen}")

발라드: 사랑..그놈
록/메탈: 사랑한 후에
랩/힙합: Get It In (Feat. Beenzino)
댄스: Hot Issue
R&B/Soul: Missing You Mean Everything) (Feat. Beenzino)
인디음악: 너를 품에 안으면
국내드라마: 너를 위해
포크/블루스: 사랑한 후에
일렉트로니카: 사랑한후에 (Feat. Beenzino)
성인가요/트로트: 내사랑 그대여
트로트: 사랑아
